# Импорты

In [6]:
import requests
import pandas as pd
import numpy as np
import json
import time
import random

from datetime import datetime        
from tqdm import tqdm
from bs4 import BeautifulSoup

# COOKIES и Параметры

In [2]:
# Общие для всех запрсов.
cookies = {
   # Ваши cookies.
}

headers = {
   # Ваш headers.
}
# Параметр для получения IDs товаров.
params1 = {
    'categoryId': '205',
    'offset': '0',
    'limit': '24',
    'filterParams': 'WyJ0b2xrby12LW5hbGljaGlpIiwiIiwiZGEiXQ==',
    'doTranslit': 'true',
}

# Параметр для получения характеристик и отзывов
params2 = {
    'productId': '42',
}

# Параметр для получения цены
params3 = {
    'productIds': '42',
    'isPromoApplied': 'true',
    'addBonusRubles': 'true',
}

# Запросы

In [3]:
# Запрос для получения IDs товаров --> json
def get_product_ids(params, cookies, headers):
    response = requests.get('https://www.mvideo.ru/bff/products/listing',
                            params=params,
                            cookies=cookies,
                            headers=headers).json()

    product_ids = response.get('body').get('products') # --> List['str'] products_IDS
    return product_ids

In [4]:
# Запрос для получения характеристик --> json
def get_details(params, cookies, headers):
    response = requests.get('https://www.mvideo.ru/bff/product-details',
                            params=params,
                            cookies=cookies,
                            headers=headers).json()
    return response

In [5]:
# Запрос для получения отзывов --> json
def get_reviews(params, cookies, headers):
    response = requests.get('https://www.mvideo.ru/bff/reviews/product', 
                            params=params, 
                            cookies=cookies,
                            headers=headers).json()
    return response

In [6]:
# Запрос для получения цены --> json
def get_prices(params, cookies, headers):
    response = requests.get('https://www.mvideo.ru/bff/products/prices',
                            params=params,
                            cookies=cookies,
                            headers=headers).json()
    return response

# Функция для сборки всех ТХ и отзывов в словарь

In [7]:
CONST_NAME_PAGE = 2808 # Последняя страница. На текущий момент 16.04.2024

In [8]:
start_page = 0 # Стартовая страница для парсинга, изменяется с шагом в 24, нумерация с нуля.
offset = 0 # Номер текущей страницы для парсинга, изменяется с шагом в 24, нумерация с нуля.
CONST_NAME_PAGE = 0 # Номер последнея страницы для парсинга, изменяется с шагом в 24, нумерация с нуля.
limit_ID = 10 # Количество продуктов на странице, изменяется с шагом в 1, нумерация с еденицы.
categoryId = '205' # ID категории товара .

dev_param1 = params1.copy() # Параметр для получения IDs товаров.
dev_param2 = params2.copy() # Параметр для получения характеристик и отзывов.
dev_param3 = params3.copy() # Параметр для получения цены.

final_dict = {} # Финальная простыня =(
all_ids_products = [] # Все айдишники товаров.

In [7]:
# Функция вернет словарь с техническими характеристиками и отзывами.
def get_all_information(final_dict, all_ids_products, param1, param2, param3, 
                 start_page=0, last_page=2808, 
                 offset=0, category_id='205', await_time=10):
    try:
        param1['categoryId'] = category_id
        try:
            while start_page <= last_page:
                product_ids = []
                param1['offset'] = str(offset)
                product_ids = get_product_ids(param1, cookies, headers)

                for id in tqdm(product_ids):
                    if id not in all_ids_products:
                        param2['productId'] = id
                        param3['productIds'] = id
                        product_details = get_details(param2, cookies, headers)
                        prices = get_prices(param3, cookies, headers)
                        reviews = get_reviews(param2, cookies, headers)
                        all_ids_products.append(id)
                        final_dict[id] = {'product_details': product_details, 'reviews': reviews, 'prices': prices}
                    time.sleep(np.random.randint(0, 3))
                offset += 24
                start_page += 24
            return final_dict, all_ids_products
        except Exception as e:
            with open('error_log.txt', mode='a', encoding='utf-8') as file:
                err = f"""
                {'==' * 50}
                ERROR-TIME: {str(datetime.now())} 
                {e}
                {'==' * 50}
                """
                file.write(err)
            print(f"TAKE ERROR COME TO SLEEP ON: {await_time} sec...")
            time.sleep(await_time)
            final_dict, all_ids_products = get_all_information(
                final_dict,
                all_ids_products,
                param1,
                param2,
                param3, 
                start_page=start_page,
                last_page=last_page,
                offset=offset, 
                category_id=category_id
            )
        return final_dict, all_ids_products
    except KeyboardInterrupt:
        return final_dict, all_ids_products

# Функции извлекающие из словаря нужное

In [10]:
# Функция вернет словарь без шелухи.
def pars_product_details(dict_prod, ids):
    norm_dict = {}

    for id in tqdm(ids):
        norm_dict[id] = {'brandName': dict_prod.get(id).get('product_details')
                         .get('body').get('brandName')}

        norm_dict[id].update({'modelName': dict_prod.get(id)
                         .get('product_details')
                         .get('body').get('modelName')})

        norm_dict[id].update({'basePrice': dict_prod.get(id).get('prices').get('body')
                              .get('materialPrices')[0].get('price').get('basePrice'),
                             'salePrice': dict_prod.get(id).get('prices').get('body')
                              .get('materialPrices')[0].get('price').get('salePrice')})

        for d in dict_prod.get(id).get('product_details').get('body').get('properties').get('all'):
            norm_dict[id].update({d.get('properties')[0].get('name'): d.get('properties')[0].get('value')})
    return norm_dict

In [11]:
# Функция вернет словарь с отзывами.
def pars_reviews(dict_prod, ids):
    norm_dict = {}

    for id in tqdm(ids):
        norm_dict[id] = {}
        for rev in dict_prod.get(id).get('reviews').get('body').get('reviews'):
            norm_dict[id].update({rev.get('reviewId'): {'date': rev.get('date'), 'score': rev.get('score'), 
                                                       'benefits': rev.get('benefits'), 'drawbacks': rev.get('drawbacks'),
                                                       'text': rev.get('text'), 'name': rev.get('name'), 
                                                       'scores': rev.get('scores')}})
    return norm_dict

In [12]:
# Функция вернет словарь отзывов без шелухи.
def get_fin_rev_dict(d_rev, ids):
    norm_dict = {}
    for id in tqdm(ids):
        for rev_id in d_rev[id]:
            if len(d_rev.get(id).get(rev_id).get('scores')) != 0:
                norm_dict[rev_id] = {'product_id': id, 'date': d_rev.get(id).get(rev_id).get('date'),
                                    'score': d_rev.get(id).get(rev_id).get('score'),
                                    'name': d_rev.get(id).get(rev_id).get('name'),
                                    'benefits': d_rev.get(id).get(rev_id).get('benefits'),
                                    'drawbacks': d_rev.get(id).get(rev_id).get('drawbacks'),
                                    'text': d_rev.get(id).get(rev_id).get('text'),
                                    'Power': d_rev.get(id).get(rev_id).get('scores')[0].get('value'),
                                    'Working_hours': d_rev.get(id).get(rev_id).get('scores')[1].get('value'),
                                    'Functionality': d_rev.get(id).get(rev_id).get('scores')[2].get('value')
                                    }
            else:
                norm_dict[rev_id] = {'product_id': id, 'date': d_rev.get(id).get(rev_id).get('date'),
                                    'score': d_rev.get(id).get(rev_id).get('score'),
                                    'name': d_rev.get(id).get(rev_id).get('name'),
                                    'benefits': d_rev.get(id).get(rev_id).get('benefits'),
                                    'drawbacks': d_rev.get(id).get(rev_id).get('drawbacks'),
                                    'text': d_rev.get(id).get(rev_id).get('text'),
                                    'Power': None,
                                    'Working_hours': None,
                                    'Functionality': None
                                    }
        
    return norm_dict

# Тест и прогон

In [13]:
# Тестовый запрос для проверки.
test = get_product_ids(params1, cookies, headers)
test

['30066317',
 '30069599',
 '400194406',
 '400171626',
 '400246909',
 '400284088',
 '400257859',
 '400269112',
 '400259093',
 '400218233',
 '30069509',
 '30063534']

# Получаем общий словарь

In [8]:
final_dict = {}
all_ids_products = []

In [ ]:
final_dict, all_ids_products = get_all_information(
    final_dict,
    all_ids_products,
    dev_param1,
    dev_param2,
    dev_param3,
    last_page=100 * 24,
    offset=0
)

# Получаем словарь и DF ТХ

In [63]:
dict_fin = pars_product_details(final_dict, ids)

100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 12009.46it/s]


In [64]:
df_det = pd.DataFrame.from_dict(dict_fin, orient='index')

# Получаем словарь и DF отзывов

In [65]:
rev_d = pars_reviews(final_dict, ids)

100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 3997.11it/s]


In [66]:
reviews = get_fin_rev_dict(rev_d, ids)

100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 2399.37it/s]


In [67]:
df_rev = pd.DataFrame.from_dict(reviews, orient='index')

# Смотрим результат

In [68]:
df_det.shape

(12, 43)

In [69]:
df_det.head(3)

,brandName,modelName,basePrice,salePrice,Гарантия,Серия,Обратите внимание,Операционная система,Экран,Процессор,...,Разрешение экрана,Тип процессора,Количество основных камер,Сенсор распознавания лица,Кабель Type-C - Type-C,Поддержка беспроводной зарядки,Обратная беспроводная зарядка,В комплекте,Время в режиме разговора,Важная особенность
400256055,HONOR,X9b 8/256GB Green,32999,28999,1 год,Honor X9b,мощный аккумулятор 5800мАч,Android MagicOS 7.2,"6.78""/1200x2652 Пикс",Qualcomm Snapdragon 6 gen1 4 x 2.2ГГц + 4 x 1....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400257033,Tecno,SPARK 20 8/128GB Gravity Black,13999,10499,1 год,Spark,NaN,Android 13,"6.56""/720x1612 Пикс",Mediatek,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30067386,Samsung,Galaxy A34 128GB Awesome Violet,29999,24999,1 год,NaN,NaN,Android 13,"6.6""/2340x1080 Пикс",Mediatek Dimensity 1080 2.6 ГГц,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df_rev.shape

(1127, 10)

In [71]:
df_rev.head(3)

,product_id,date,score,name,benefits,drawbacks,text,Power,Working_hours,Functionality
r61745873,400256055,2024-04-08,5.0,Анна,"Красивый, яркий экран. Моментальная реакция на...",Не нашла,Лучше айфона)),5.0,5.0,5.0
r61715430,400256055,2024-03-16,5.0,Vladimir,Аккумулятор. Изогнутый экран. Противоударное с...,Не выявил,None,5.0,5.0,5.0
r61704733,400256055,2024-03-13,5.0,вера,Камера супер. Видео супер.,Минусов нет.,Очень понравился товар. Всё просто супер. Каче...,5.0,5.0,5.0


In [9]:
# 1. Будет реализован автоматический сбор всех данных в DF.
# 2. Будет реализовано автоматическое генерирование cookies и headers
# 3. Будет реализована возможность выбора категории товара